In [1]:
print("HI")

HI


In [2]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.linear_model import LogisticRegression as LR
from sklearn.feature_extraction.text import TfidfVectorizer as CV
from sklearn.ensemble.gradient_boosting import GradientBoostingClassifier as GB
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier as KNC
from Database import Database
from numpy import array
from sklearn.naive_bayes import MultinomialNB as NB
import sqlite3
import tqdm
import random

In [3]:
conn = sqlite3.connect('berita.db')
c = conn.cursor()
c.execute("SELECT Text,Date,Clock,Sentiment From Berita ")
result = c.fetchall()

In [4]:
d = Database()
data = {}
label = {}
for I in result:
    session = d.cariSesi(I[1],I[2])[0]
    if session not in data:
        data[session] = []
        label[session] = I[3]

In [5]:
# Sorting the chronological order

#MeTooLazySoMeBubbleSort

chronology = list(data.keys())

for I in range(len(data.keys())):
    for J in range(I+1,len(data.keys())):
        if chronology[I] > chronology[J]:
            chronology[I],chronology[J] = chronology[J],chronology[I]

In [6]:
date = [chronology[len(chronology)*I//6-1] for I in range(1,7)]

In [7]:
def balancedTrain(X,y,mode):
    '''
    mode has two options:
    -> 'CV'
    then balancedTrain would suit it undersampling method for 
    cross validation
    
    -> 'Boosting'
    then balancedTrain would suit is sampling method for
    boosting    
    '''
    assert mode == 'CV' or mode == 'Boosting'
    balancedX = []
    balancedY = []
    
    index = {}
    
    assert len(X) == len(y)
    
    #Mencatat indeks mana yang positif dan mana yang negatif
    for I in range(len(X)):
        if y[I] not in index:
            index[y[I]] = []
        index[y[I]].append(I)
    
    minimumPoint = min([len(I) for I in index.values()])    
    
    #Memastikan jumlah (+) dan (-) sama
    for I in index:
        if mode == 'CV':
            chosen = random.sample(index[I],minimumPoint)
        else:
            chosen = random.choices(index[I],k=minimumPoint)
        for J in chosen:
            balancedX.append(X[J])
            balancedY.append(y[J])
    return balancedX, balancedY

In [8]:
def transform(text):
    listNews = []
    for news in text:
        listSentence = []
        oldText = news
        while True:
            newText = oldText.replace("  "," ")
            if oldText == newText:
                break
            oldText = newText
        listSentence.extend(newText)
        splitText = newText.split(" ")
        listSentence.append(" ".join([splitText[I] for I in range(0,len(splitText),2)]))
        listSentence.append(" ".join([splitText[I] for I in range(1,len(splitText),2)]))
        listNews.append(" SNIP ".join(listSentence))
    return listNews

In [9]:
def splitGroup(kFold):
    #Me too lazy to plug in sklearn
    assert len(X) == len(y)
    index = [int(I) for I in range(len(X))]
    random.shuffle(index)
    group = [index[len(index)*I//kFold:len(index)*(I+1)//kFold] for I in range(kFold)]
    return group

In [10]:
#Custom class for GradientBoosting
class Boosting():
    def __init__(self):
        self.clf = GB()
    def fit(self,X,y):
        self.clf.fit(X,y)
    def predict(self,X):
        m = int(X.shape[0] ** (0.5))
        pred = []
        for I in range(m):
            pred.extend(self.clf.predict(X[I*X.shape[0]//m:(I+1)*X.shape[0]//m].toarray()))
        return pred
#Custom class for K Nearest Neighbor
class Neighbors:
    def __init__(self):
        self.clf = KNC()
    def fit(self,X,y):
        self.clf.fit(X,y)
    def predict(self,X):
        m = int(X.shape[0] ** (0.5))
        pred = []
        for I in range(m):
            pred.extend(self.clf.predict(X[I*X.shape[0]//m:(I+1)*X.shape[0]//m]))
        return pred

In [11]:
def bersihkanTeksBerita(array):
    hasil = []
    for news in array:
        newSentence = ""
        for J in news:
            if J.isalpha():
                newSentence += J
            else:
                newSentence += " "
        hasil.append(newSentence)
    return hasil

In [12]:
clfOption = [Boosting(),LR(n_jobs = -1),NB(),LinearSVC(),Neighbors(),RFC()]
mrePred = []

In [13]:
def mrc(pred,Y):
    assert len(pred)==len(Y)
    pred = array(pred)
    Y    = array(Y)
    
    TP, FP , TN, FN = 0,0,0,0
    
    for I in range(len(pred)):
        if pred[I] == Y[I]:
            if pred[I] == 1:
                TP += 1
            else:
                TN += 1
        else:
            if pred[I] == -1:
                FP += 1
            else:
                FN += 1
    print(TP,FP,TN,FN)
    try:
        return ((TP*TN)-(FP*FN)) / ((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))**(0.5)
    except:
        return 0

In [14]:
del result
del data

In [15]:
print("Mulai")

Mulai


In [ ]:
mrePred = []
for iter in range(5):
    print("Memulai pengambilan data")
    mreTotal = []
    query = "Select * from berita WHERE Date <= "+str(date[iter])
    c.execute(query)
    trainData = c.fetchall()
    
    
    query = "Select * from berita WHERE Date <= "+str(date[iter+1])+" AND "+str(date[iter])
    c.execute(query)
    testData = c.fetchall()
    # Do cross-validation to choose the best feature selection
    
    X = bersihkanTeksBerita([I[3] for I in trainData])
    y = [I[7] for I in trainData]
    
    # Mengurangi memori
    del trainData
    
    print("Memulai CV")       
    group = splitGroup(5)
    XkFold = [[X[J] for J in K]for K in group]
    YkFold = [[y[J] for J in K]for K in group]
    counterList = []
    selectList = []
    mreTotal = []
  
    for I in range(5):
        xTrain = []
        yTrain = []
    
        xTest  = []
        yTest = []
        
        for J in range(5):
            if J != I:
                xTrain.extend(XkFold[J])
                yTrain.extend(YkFold[J])
            else:
                testIndex = J

        xTrain = transform(xTrain)
        xTest = transform(XkFold[testIndex])

        assert len(xTrain) == len(yTrain)
        xTrainNew , yTrainNew = balancedTrain(xTrain,yTrain,'CV')
        counterList.append(CV(ngram_range=(2,2),min_df=5)) 
        trainVector = counterList[-1].fit_transform(xTrainNew)
        testVector  = counterList[-1].transform(xTest)

        selectList.append(SelectKBest(chi2, k = min(10000,trainVector.shape[1])))

        trainVector = selectList[-1].fit_transform(trainVector,yTrainNew)
        testVector  = selectList[-1].transform(testVector)

        mreTotal.append(0)
        for J in clfOption:
            J.fit(trainVector,yTrainNew)
            prediction = J.predict(testVector)
            mreTotal[-1] += mrc(prediction,YkFold[testIndex])
    
    index = mreTotal.index(max(mreTotal))
    
    mrePred.append({'post':[],
                    'chronological':[]})
    
    xTrainNew = []
    yTrainNew = []
    # Generating boosting data
    for I in range(5):
        X_temp , y_temp = balancedTrain(xTrain,yTrain,'Boosting')
        xTrainNew.append(X_temp)
        yTrainNew.append(y_temp)

    trainVector = [counterList[index].transform(I) for I in xTrainNew]        
    trainVector = [selectList[index].transform(I)for I in trainVector]
        
    # Create the test set of chronological entries
        
    lengthOfTestData = {}
    testX = []
    dataY = {}

    for entry in testData:
        if entry[5] not in lengthOfTestData:
            lengthOfTestData[entry[5]] = 0
            dataY[entry[5]] = entry[7]
        lengthOfTestData[entry[5]] += 1
        testX.append(entry[3])
    
    testX = counterList[index].transform(transform(testX))
    testVector = selectList[index].transform(testX)
    print("Mulai training")
    for I in clfOption:
        postPredict = array([int(0) for J in range(testVector.shape[0])])
        for boostingIter in range(5):
            I.fit(trainVector[boostingIter],yTrainNew[boostingIter])
            postPredict += I.predict(testVector)
        postPredict = [[-1,1][J>0] for J in postPredict]

        dayPredict = []
        postY = []
        dayY = []
        sumIndex = 0

        for dateTested in lengthOfTestData:               
            dayPredict.append([-1,1][sum(postPredict[sumIndex:sumIndex+lengthOfTestData[dateTested]])>0])
            postY.extend([dataY[dateTested] for I in range(lengthOfTestData[dateTested])])
            dayY.append(dataY[dateTested])
            sumIndex += lengthOfTestData[dateTested]

        mrePred[-1]['post'].append(mrc(postPredict,postY))
        mrePred[-1]['chronological'].append(mrc(dayPredict,dayY))
            

Memulai pengambilan data
Memulai CV
710 1422 1195 291
1824 308 346 1140
1102 1030 1047 439
1752 380 450 1036
853 1279 1021 465
1207 925 943 543
741 1369 1175 333
1762 348 367 1141
1125 985 993 515
1670 440 478 1030
1690 420 411 1097
1191 919 951 557
721 1471 1195 231
1899 293 279 1147
1169 1023 983 443
1822 370 397 1029
1692 500 491 935
1263 929 908 518
703 1476 1191 248
1879 300 278 1161
1139 1040 983 456
1785 394 376 1063
1327 852 720 719
1244 935 928 511
664 1475 1221 259
1834 305 257 1223
1051 1088 1033 447
1791 348 356 1124
1950 189 210 1270
1201 938 927 553
Mulai training
7109 14501 8015 3798
22 134 77 11
20389 1221 786 11027
156 0 0 88
10781 10829 6397 5416
75 81 57 31
19479 2131 1355 10458
156 0 0 88
18804 2806 1638 10175
155 1 0 88
13255 8355 4818 6995
128 28 26 62
Memulai pengambilan data
Memulai CV
1421 2918 1830 515
3482 857 651 1694
2185 2154 1530 815
3334 1005 790 1555
2901 1438 1015 1330
2342 1997 1457 888
3155 1146 855 1529
3443 858 691 1693
2007 2294 1643 741
3285 1016

In [16]:
name = ["Gradient Boosting","Logistic Regression","Naive Bayes","Linear SVC","K nearest neighbor","Random forest"]
for I in range(len(name)):
    chronological = [J['chronological'][I] for J in mrePred]
    post = [J['post'][I] for J in mrePred]
    print("%s -> (chronological) %f (post) %f " % (name[I],sum(chronological)/5,sum(post)/5))

Gradient Boosting -> (chronological) 0.000259 (post) 0.013011 
Logistic Regression -> (chronological) 0.008833 (post) 0.019391 
Naive Bayes -> (chronological) 0.018590 (post) 0.019025 
Linear SVC -> (chronological) 0.000854 (post) 0.019949 
K nearest neighbor -> (chronological) 0.045772 (post) 0.015734 
Random forest -> (chronological) 0.028655 (post) 0.017214 


In [17]:
mrePred

[{'chronological': [-0.08041741036528247,
   -0.00903425307795571,
   0.060263040633680896,
   -0.012940520237177025,
   0.07719184171054305,
   0.03357044648161351],
  'post': [0.004039105992426345,
   0.035046231501828444,
   0.03866386321371173,
   0.03893165512341328,
   0.011314922078974471,
   0.02493921602405859]},
 {'chronological': [-0.029912249562631926,
   -0.03552317266337992,
   -0.0263179260078171,
   -0.04023941963382059,
   -0.04536780536262693,
   -0.05792269894721858],
  'post': [-0.02259194834389064,
   -0.0006551549366040136,
   -0.014682214448911015,
   -0.002157793701502563,
   0.008281815485184486,
   -0.028418858537299908]},
 {'chronological': [-0.01611601285437681,
   0.02375866681202409,
   0.01581961698419295,
   -0.002935163924097879,
   0.1504460220535426,
   0.08855539127335335],
  'post': [0.02280172255273168,
   0.003309469206891995,
   0.013050911297837453,
   0.003592007967474261,
   0.03169339785935832,
   0.034775158053151976]},
 {'chronological': [0

In [ ]:
for I in clfOption:
    postPredict = array([int(0) for J in range(testVector.shape[0])])
    for boostingIter in range(5):
        I.fit(trainVector[boostingIter],yTrainNew[boostingIter])
        postPredict += I.predict(testVector)
    postPredict = [[-1,1][J>0] for J in postPredict]
        
    dayPredict = []
    postY = []
    dayY = []
    sumIndex = 0
    
    for date in lengthOfTestData:               
        dayPredict.append([-1,1][sum(postPredict[sumIndex:sumIndex+lengthOfTestData[date]])>0])
        postY.extend([dataY[date] for I in range(lengthOfTestData[date])])
        dayY.append(dataY[date])
        sumIndex += lengthOfTestData[date]
            
    mrePred[-1]['post'].append(mrc(postPredict,postY))
    mrePred[-1]['chronological'].append(mrc(dayPredict,dayY))

In [ ]:
print("Mulai training")
    for I in clfOption:
        postPredict = array([int(0) for J in range(testVector.shape[0])])
        for boostingIter in range(5):
            I.fit(trainVector[boostingIter],yTrainNew[boostingIter])
            postPredict += I.predict(testVector)
        postPredict = [[-1,1][J>0] for J in postPredict]
        
        dayPredict = []
        postY = []
        sumIndex = 0
    
        for date in entry:               
            dayPredict.append([-1,1][sum(postPredict[sumIndex:sumIndex+len(lengthOfTestData[date])])>0])
            postY.extend([label[J] for I in range(len(lengthOfTestData[date]))])
            dayY.append(label[J])
            sumIndex += len(data[J])
            
        mrePred[-1]['post'].append(mrc(postPredict,postY))
        mrePred[-1]['chronological'].append(mrc(dayPredict,dayY))

In [ ]:
trainVector.shape[1]

In [ ]:
testX[0]